In [37]:
!pip install spacy

In [38]:
!python3 -m spacy download en_core_web_sm


     |████████████████████████████████| 13.9 MB 124 kB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [39]:
import pandas as pd
import fractions
import re
import string
import nltk
import spacy 
import numpy as np
import math
from tqdm import tqdm 

from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
pd.set_option('display.max_colwidth',200)


data=pd.read_csv("amazon_baby.csv",error_bad_lines=False)
data

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion not worth keeping. I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it looks...fit the full size bed perfectly...would recommend to anyone looking for this type of quilt,5
3,Stop Pacifier Sucking without tears with Thumbuddy To Love's Binky Fairy Puppet and Adorable Book,"This is a product well worth the purchase. I have not found anything else like this, and it is a positive, ingenious approach to losing the binky. What I love most about this product is how much...",5
4,Stop Pacifier Sucking without tears with Thumbuddy To Love's Binky Fairy Puppet and Adorable Book,"All of my kids have cried non-stop when I tried to ween them off their pacifier, until I found Thumbuddy To Love's Binky Fairy Puppet. It is an easy way to work with your kids to allow them to un...",5
...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Shaped Pendant (Smokey Black),"Such a great idea! very handy to have and look really cool too, not to mention great for baby's gums to help reduce the pain of teething!",5
183527,Baby Teething Necklace for Mom Pretty Donut Shaped Pendant (Smokey Black),This product rocks! It is a great blend of function and fashion. Aside from the unique concept of having something easily accessible and cool for baby to play and soothe themselves as cool acces...,5
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids....I know this Company and its great in quality...,5
183529,"Baby Food Freezer Tray - Bacteria Resistant, BPA and Phthalate Free - FREE Recipe E-book - Silicone Storage Container with Lid for Homemade Baby Food - 9 Easy-Out Portions - Lifetime Guarantee - P...",I am extremely happy with this product. I have been searching for a product like this for a long time and just not found the right one. I am a mum who is very into my organic products and unllike ...,5


In [40]:
!pip3 install Pystemmer 

In [41]:
nlp=spacy.load('en_core_web_sm')


In [42]:
data_text=data[["review"]]
data_text['index']=data_text.index

documents=data_text
documents



<ipython-input-42-1da49a032119>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index']=data_text.index


,review,index
0,"These flannel wipes are OK, but in my opinion not worth keeping. I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher ...",0
1,it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.,1
2,Very soft and comfortable and warmer than it looks...fit the full size bed perfectly...would recommend to anyone looking for this type of quilt,2
3,"This is a product well worth the purchase. I have not found anything else like this, and it is a positive, ingenious approach to losing the binky. What I love most about this product is how much...",3
4,"All of my kids have cried non-stop when I tried to ween them off their pacifier, until I found Thumbuddy To Love's Binky Fairy Puppet. It is an easy way to work with your kids to allow them to un...",4
...,...,...
183526,"Such a great idea! very handy to have and look really cool too, not to mention great for baby's gums to help reduce the pain of teething!",183526
183527,This product rocks! It is a great blend of function and fashion. Aside from the unique concept of having something easily accessible and cool for baby to play and soothe themselves as cool acces...,183527
183528,This item looks great and cool for my kids....I know this Company and its great in quality...,183528
183529,I am extremely happy with this product. I have been searching for a product like this for a long time and just not found the right one. I am a mum who is very into my organic products and unllike ...,183529


In [43]:
documents["review"][4]

"All of my kids have cried non-stop when I tried to ween them off their pacifier, until I found Thumbuddy To Love's Binky Fairy Puppet.  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from it.This is a must buy book, and a great gift for expecting parents!!  You will save them soo many headaches.Thanks for this book!  You all rock!!"

In [44]:
documents["review"]


0         These flannel wipes are OK, but in my opinion not worth keeping.  I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher ...
1                                                  it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.
2                                                                 Very soft and comfortable and warmer than it looks...fit the full size bed perfectly...would recommend to anyone looking for this type of quilt
3         This is a product well worth the purchase.  I have not found anything else like this, and it is a positive, ingenious approach to losing the binky.  What I love most about this product is how much...
4         All of my kids have cried non-stop when I tried to ween them off their pacifier, until I found Thumbuddy To Love's Binky Fairy Puppet.  It is an easy 

In [45]:
used_doc=documents[:57000]

In [46]:
used_doc

,review,index
0,"These flannel wipes are OK, but in my opinion not worth keeping. I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher ...",0
1,it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.,1
2,Very soft and comfortable and warmer than it looks...fit the full size bed perfectly...would recommend to anyone looking for this type of quilt,2
3,"This is a product well worth the purchase. I have not found anything else like this, and it is a positive, ingenious approach to losing the binky. What I love most about this product is how much...",3
4,"All of my kids have cried non-stop when I tried to ween them off their pacifier, until I found Thumbuddy To Love's Binky Fairy Puppet. It is an easy way to work with your kids to allow them to un...",4
...,...,...
56995,I bought this bottle when I was pregnant with my now 7 month old son. I purchased them because they were BPA free and they were suppose to be good for a baby who is both breast fed and bottle fed....,56995
56996,I buy them for being orthodontic but still my baby has not wanted to use. Even I could not convince her to use any kind of nipple,56996
56997,My baby has a strong suction and ends up gulping a lot of air with the slow flow nipples. She is 3 weeks old and is switching between breast and this bottle. I can hear the sound of milk flow pl...,56997
56998,This is a cute bottle but not my baby's favorite. She prefers Dr. Brown's Natural Flow Feeding Bottles or Playtex VentAire Bottles -- both of which are available on Amazon.,56998


In [47]:
used_doc.dropna(subset = ["review"], inplace=True) # drop those rows which have NaN value cells

<ipython-input-47-ada661c3bb41>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_doc.dropna(subset = ["review"], inplace=True) # drop those rows which have NaN value cells


In [48]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk.stem as stemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/deth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [49]:
stemmer=PorterStemmer()
def lemmatize_stemming(text):
      return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3:
              result.append(lemmatize_stemming(token))
  
    return result

In [50]:
doc_sample=used_doc[used_doc['index']==4].values[0][0]
print('original document')
words=[]
for word in doc_sample.split(' '):
      words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document
['All', 'of', 'my', 'kids', 'have', 'cried', 'non-stop', 'when', 'I', 'tried', 'to', 'ween', 'them', 'off', 'their', 'pacifier,', 'until', 'I', 'found', 'Thumbuddy', 'To', "Love's", 'Binky', 'Fairy', 'Puppet.', '', 'It', 'is', 'an', 'easy', 'way', 'to', 'work', 'with', 'your', 'kids', 'to', 'allow', 'them', 'to', 'understand', 'where', 'their', 'pacifier', 'is', 'going', 'and', 'help', 'them', 'part', 'from', 'it.This', 'is', 'a', 'must', 'buy', 'book,', 'and', 'a', 'great', 'gift', 'for', 'expecting', 'parents!!', '', 'You', 'will', 'save', 'them', 'soo', 'many', 'headaches.Thanks', 'for', 'this', 'book!', '', 'You', 'all', 'rock!!']


 tokenized and lemmatized document: 
['kid', 'cri', 'stop', 'tri', 'ween', 'pacifi', 'thumbuddi', 'love', 'binki', 'fairi', 'puppet', 'easi', 'work', 'kid', 'allow', 'understand', 'pacifi', 'go', 'help', 'book', 'great', 'gift', 'expect', 'parent', 'save', 'headach', 'thank', 'book', 'rock']


In [51]:
processed_docs = used_doc['review'].map(preprocess)
processed_docs[:10]

0    [flannel, wipe, opinion, worth, keep, order, someims, vims, cloth, wipe, ocean, blue, countwhich, larger, nicer, softer, textur, higher, qualiti, cloth, wipe, hand, face, usingthirsti, pack, wipe,...
1                                                                                       [come, earli, disappoint, love, planet, wise, bag, wipe, holder, kep, osocozi, wipe, moist, leak, highli, recommend]
2                                                                                                                               [soft, comfort, warmer, look, size, perfectli, recommend, look, type, quilt]
3             [product, worth, purchas, like, posit, ingeni, approach, lose, binki, love, product, ownership, daughter, get, binki, proud, love, littl, fairi, love, artwork, chart, clever, approach, tool]
4      [kid, cri, stop, tri, ween, pacifi, thumbuddi, love, binki, fairi, puppet, easi, work, kid, allow, understand, pacifi, go, help, book, great, gift, expect, parent, save, hea

In [52]:
test=processed_docs[4]
test

['kid',
 'cri',
 'stop',
 'tri',
 'ween',
 'pacifi',
 'thumbuddi',
 'love',
 'binki',
 'fairi',
 'puppet',
 'easi',
 'work',
 'kid',
 'allow',
 'understand',
 'pacifi',
 'go',
 'help',
 'book',
 'great',
 'gift',
 'expect',
 'parent',
 'save',
 'headach',
 'thank',
 'book',
 'rock']

In [53]:
dictionary=gensim.corpora.Dictionary(processed_docs)
count=0

for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count>10:
        break

0 blue
1 boyfor
2 cloth
3 countwhich
4 face
5 flannel
6 hand
7 handl
8 higher
9 issu
10 keep


In [54]:
wordfreq = []
for w in test:
    wordfreq.append(test.count(w))


print("Pairs\n" + str(list(zip(test, wordfreq))))

Pairs
[('kid', 2), ('cri', 1), ('stop', 1), ('tri', 1), ('ween', 1), ('pacifi', 2), ('thumbuddi', 1), ('love', 1), ('binki', 1), ('fairi', 1), ('puppet', 1), ('easi', 1), ('work', 1), ('kid', 2), ('allow', 1), ('understand', 1), ('pacifi', 2), ('go', 1), ('help', 1), ('book', 2), ('great', 1), ('gift', 1), ('expect', 1), ('parent', 1), ('save', 1), ('headach', 1), ('thank', 1), ('book', 2), ('rock', 1)]


In [55]:
print(max(wordfreq))

2


In [56]:
if max(wordfreq):
    print("Pairs\n" + str(list(zip(test, wordfreq))))
        

Pairs
[('kid', 2), ('cri', 1), ('stop', 1), ('tri', 1), ('ween', 1), ('pacifi', 2), ('thumbuddi', 1), ('love', 1), ('binki', 1), ('fairi', 1), ('puppet', 1), ('easi', 1), ('work', 1), ('kid', 2), ('allow', 1), ('understand', 1), ('pacifi', 2), ('go', 1), ('help', 1), ('book', 2), ('great', 1), ('gift', 1), ('expect', 1), ('parent', 1), ('save', 1), ('headach', 1), ('thank', 1), ('book', 2), ('rock', 1)]


In [57]:
test_text=nlp(doc_sample)




In [58]:
for tok in test_text:
    print(tok.text,'->',tok.dep_,'->',tok.pos_)

All -> nsubj -> PRON
of -> prep -> ADP
my -> poss -> PRON
kids -> pobj -> NOUN
have -> aux -> AUX
cried -> ROOT -> VERB
non -> dobj -> ADJ
- -> xcomp -> ADJ
stop -> dobj -> NOUN
when -> advmod -> SCONJ
I -> nsubj -> PRON
tried -> advcl -> VERB
to -> aux -> PART
ween -> xcomp -> VERB
them -> dobj -> PRON
off -> prep -> ADP
their -> poss -> PRON
pacifier -> pobj -> NOUN
, -> punct -> PUNCT
until -> mark -> SCONJ
I -> nsubj -> PRON
found -> advcl -> VERB
Thumbuddy -> dobj -> PROPN
To -> prep -> ADP
Love -> poss -> PROPN
's -> case -> PART
Binky -> compound -> PROPN
Fairy -> compound -> PROPN
Puppet -> pobj -> PROPN
. -> punct -> PUNCT
  -> dep -> SPACE
It -> nsubj -> PRON
is -> ROOT -> AUX
an -> det -> DET
easy -> amod -> ADJ
way -> attr -> NOUN
to -> aux -> PART
work -> relcl -> VERB
with -> prep -> ADP
your -> poss -> PRON
kids -> pobj -> NOUN
to -> aux -> PART
allow -> advcl -> VERB
them -> nsubj -> PRON
to -> aux -> PART
understand -> ccomp -> VERB
where -> advmod -> SCONJ
their -> po

In [59]:
for tok in test_text:
    if tok.pos_=='NOUN':
        print(tok.text,'->',tok.dep_,'->',tok.pos_)

kids -> pobj -> NOUN
stop -> dobj -> NOUN
pacifier -> pobj -> NOUN
way -> attr -> NOUN
kids -> pobj -> NOUN
pacifier -> nsubj -> NOUN
part -> dobj -> NOUN
book -> dobj -> NOUN
gift -> conj -> NOUN
parents -> dobj -> NOUN
soo -> dobj -> NOUN
headaches -> dobj -> NOUN
Thanks -> ROOT -> NOUN
book -> pobj -> NOUN
rock -> ROOT -> NOUN


In [61]:
noun=[]
for tok in test_text:
    if tok.pos_=='NOUN':
        print(tok.text,'->',tok.pos_)
        noun.append(tok.text)

kids -> NOUN
stop -> NOUN
pacifier -> NOUN
way -> NOUN
kids -> NOUN
pacifier -> NOUN
part -> NOUN
book -> NOUN
gift -> NOUN
parents -> NOUN
soo -> NOUN
headaches -> NOUN
Thanks -> NOUN
book -> NOUN
rock -> NOUN


In [62]:
noun

['kids',
 'stop',
 'pacifier',
 'way',
 'kids',
 'pacifier',
 'part',
 'book',
 'gift',
 'parents',
 'soo',
 'headaches',
 'Thanks',
 'book',
 'rock']

In [63]:
word_freq = []
for w in noun:
    word_freq.append(noun.count(w))


print("Pairs\n" + str(list(zip(noun, word_freq))))

Pairs
[('kids', 2), ('stop', 1), ('pacifier', 2), ('way', 1), ('kids', 2), ('pacifier', 2), ('part', 1), ('book', 2), ('gift', 1), ('parents', 1), ('soo', 1), ('headaches', 1), ('Thanks', 1), ('book', 2), ('rock', 1)]


In [69]:
test_list= (list(zip(noun, word_freq)))

In [70]:
test_list

[('kids', 2),
 ('stop', 1),
 ('pacifier', 2),
 ('way', 1),
 ('kids', 2),
 ('pacifier', 2),
 ('part', 1),
 ('book', 2),
 ('gift', 1),
 ('parents', 1),
 ('soo', 1),
 ('headaches', 1),
 ('Thanks', 1),
 ('book', 2),
 ('rock', 1)]

In [72]:
for i in test_list:
    if i[1]==max(word_freq):
        print(i[0],i[1])

kids 2
pacifier 2
kids 2
pacifier 2
book 2
book 2
